In [2]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [1]:
def get_data(path, noise=False):
    data = np.load(path)
    if noise == True :
        data = data + np.random.normal(0, 0.001, data.shape)
    return data

In [3]:
X_train = get_data('enron-train-features.pkl').astype(np.uint8)
y_train = get_data('enron-train-labels.pkl').astype(np.int8)
X_test = get_data('enron-test-features.pkl').astype(np.uint8)
y_test = get_data('enron-test-labels.pkl').astype(np.int8)

In [4]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (1123, 1001)
Train_Y:  (1123, 53)
Test_X:  (579, 1001)
Test_Y:  (579, 53)


In [7]:
def hamming_accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

## Bidirectional Model


In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.backend.tensorflow_backend import clear_session
from sklearn.model_selection import train_test_split

In [9]:
ENS_COUNT = 2
clear_session()

In [10]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.4))
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

In [11]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_forward = []

    #Training
    x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = x_t.copy()
    for i in range(y_t.shape[1]):
        print("Training chain node ",i)
        y_now = y_t[:,[i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=5,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_forward.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_fwd.append(chain_forward)

TRAINING ENSEMBLE 0 :



Training chain node  0
Shapes:
 X = (673, 1001) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 24s 35ms/step - loss: 0.2252 - acc: 0.9510
Epoch 2/5
673/673 [==============================] - 0s 233us/step - loss: 0.1227 - acc: 0.9837
Epoch 3/5
673/673 [==============================] - 0s 242us/step - loss: 0.0863 - acc: 0.9837
Epoch 4/5
673/673 [==============================] - 0s 233us/step - loss: 0.0622 - acc: 0.9837
Epoch 5/5
673/673 [==============================] - 0s 270us/step - loss: 0.0455 - acc: 0.9837
Training of node 0 complete


Training chain node  1
Shapes:
 X = (673, 1002) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 1s 972us/step - loss: 0.3210 - acc: 0.8915
Epoch 2/5
673/673 [==============================] - 0s 240us/step - loss: 0.2226 - acc: 0.9643
Epoch 3/5
673/673 [==============================] - 0s 240us/step - loss: 0.1439 - acc: 0.9643
Epoch 4/5
673/673 [==============================] 

673/673 [==============================] - 0s 256us/step - loss: 0.4645 - acc: 0.8232
Epoch 3/5
673/673 [==============================] - 0s 252us/step - loss: 0.3563 - acc: 0.8871
Epoch 4/5
673/673 [==============================] - 0s 240us/step - loss: 0.2295 - acc: 0.9212
Epoch 5/5
673/673 [==============================] - 0s 258us/step - loss: 0.1845 - acc: 0.9510
Training of node 14 complete


Training chain node  15
Shapes:
 X = (673, 1016) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 1s 2ms/step - loss: 0.1624 - acc: 0.9866
Epoch 2/5
673/673 [==============================] - 0s 246us/step - loss: 0.0737 - acc: 0.9955
Epoch 3/5
673/673 [==============================] - 0s 258us/step - loss: 0.0734 - acc: 0.9955
Epoch 4/5
673/673 [==============================] - 0s 252us/step - loss: 0.0690 - acc: 0.9955
Epoch 5/5
673/673 [==============================] - 0s 253us/step - loss: 0.0634 - acc: 0.9955
Training of node 15 complete


Training chain node  1

673/673 [==============================] - 0s 277us/step - loss: 0.0582 - acc: 0.9926
Epoch 4/5
673/673 [==============================] - 0s 286us/step - loss: 0.0367 - acc: 0.9926
Epoch 5/5
673/673 [==============================] - 0s 273us/step - loss: 0.0256 - acc: 0.9926
Training of node 28 complete


Training chain node  29
Shapes:
 X = (673, 1030) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 2s 3ms/step - loss: 0.3490 - acc: 0.8767
Epoch 2/5
673/673 [==============================] - 0s 326us/step - loss: 0.1742 - acc: 0.9703
Epoch 3/5
673/673 [==============================] - 0s 342us/step - loss: 0.1363 - acc: 0.9643
Epoch 4/5
673/673 [==============================] - 0s 314us/step - loss: 0.1005 - acc: 0.9747
Epoch 5/5
673/673 [==============================] - 0s 326us/step - loss: 0.0735 - acc: 0.9851
Training of node 29 complete


Training chain node  30
Shapes:
 X = (673, 1031) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 2

673/673 [==============================] - 0s 282us/step - loss: 0.0579 - acc: 0.9777
Epoch 5/5
673/673 [==============================] - 0s 282us/step - loss: 0.0432 - acc: 0.9777
Training of node 42 complete


Training chain node  43
Shapes:
 X = (673, 1044) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 3s 4ms/step - loss: 0.3474 - acc: 0.9465
Epoch 2/5
673/673 [==============================] - 0s 286us/step - loss: 0.2840 - acc: 0.9643
Epoch 3/5
673/673 [==============================] - 0s 302us/step - loss: 0.1890 - acc: 0.9658
Epoch 4/5
673/673 [==============================] - 0s 286us/step - loss: 0.1307 - acc: 0.9673
Epoch 5/5
673/673 [==============================] - 0s 308us/step - loss: 0.1125 - acc: 0.9718
Training of node 43 complete


Training chain node  44
Shapes:
 X = (673, 1045) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 3s 4ms/step - loss: 0.3995 - acc: 0.9004
Epoch 2/5
673/673 [==============================] - 0s 

In [12]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_reverse = []
    #Training
    x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = x_t.copy()
    for i in range(1,y_t.shape[1]+1):
        print("Training chain node ",i)
        y_now = y_t[:,[-i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=5,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_reverse.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_rev.append(chain_reverse)

TRAINING ENSEMBLE 0 :



Training chain node  1
Shapes:
 X = (673, 1001) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 3s 5ms/step - loss: 0.1451 - acc: 0.9896
Epoch 2/5
673/673 [==============================] - 0s 317us/step - loss: 0.0313 - acc: 0.9985
Epoch 3/5
673/673 [==============================] - 0s 326us/step - loss: 0.0268 - acc: 0.9985
Epoch 4/5
673/673 [==============================] - 0s 329us/step - loss: 0.0258 - acc: 0.9985
Epoch 5/5
673/673 [==============================] - 0s 356us/step - loss: 0.0250 - acc: 0.9985
Training of node 1 complete


Training chain node  2
Shapes:
 X = (673, 1002) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 4s 5ms/step - loss: 0.1534 - acc: 0.9673
Epoch 2/5
673/673 [==============================] - 0s 336us/step - loss: 0.0382 - acc: 0.9970
Epoch 3/5
673/673 [==============================] - 0s 378us/step - loss: 0.0335 - acc: 0.9970
Epoch 4/5
673/673 [==============================] - 0s

673/673 [==============================] - 4s 7ms/step - loss: 0.1573 - acc: 0.9777
Epoch 2/5
673/673 [==============================] - 0s 393us/step - loss: 0.0983 - acc: 0.9926
Epoch 3/5
673/673 [==============================] - 0s 323us/step - loss: 0.0846 - acc: 0.9926
Epoch 4/5
673/673 [==============================] - 0s 350us/step - loss: 0.0704 - acc: 0.9926
Epoch 5/5
673/673 [==============================] - 0s 314us/step - loss: 0.0640 - acc: 0.9926
Training of node 15 complete


Training chain node  16
Shapes:
 X = (673, 1016) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 4s 7ms/step - loss: 0.2607 - acc: 0.9495
Epoch 2/5
673/673 [==============================] - ETA: 0s - loss: 0.1591 - acc: 0.983 - 0s 304us/step - loss: 0.1671 - acc: 0.9822
Epoch 3/5
673/673 [==============================] - 0s 311us/step - loss: 0.0915 - acc: 0.9822
Epoch 4/5
673/673 [==============================] - 0s 323us/step - loss: 0.0543 - acc: 0.9822
Epoch 5/5
673/673

673/673 [==============================] - 5s 8ms/step - loss: 0.3245 - acc: 0.9138
Epoch 2/5
673/673 [==============================] - 0s 342us/step - loss: 0.2128 - acc: 0.9480
Epoch 3/5
673/673 [==============================] - 0s 345us/step - loss: 0.1488 - acc: 0.9480
Epoch 4/5
673/673 [==============================] - 0s 353us/step - loss: 0.1011 - acc: 0.9525
Epoch 5/5
673/673 [==============================] - 0s 311us/step - loss: 0.0678 - acc: 0.9658
Training of node 29 complete


Training chain node  30
Shapes:
 X = (673, 1030) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 6s 9ms/step - loss: 0.3518 - acc: 0.9094
Epoch 2/5
673/673 [==============================] - 0s 323us/step - loss: 0.2437 - acc: 0.9391
Epoch 3/5
673/673 [==============================] - 0s 344us/step - loss: 0.1852 - acc: 0.9391
Epoch 4/5
673/673 [==============================] - 0s 313us/step - loss: 0.1424 - acc: 0.9391
Epoch 5/5
673/673 [==============================] - 0s

673/673 [==============================] - 0s 350us/step - loss: 0.1212 - acc: 0.9911
Epoch 3/5
673/673 [==============================] - 0s 322us/step - loss: 0.0987 - acc: 0.9911
Epoch 4/5
673/673 [==============================] - 0s 342us/step - loss: 0.0620 - acc: 0.9911
Epoch 5/5
673/673 [==============================] - 0s 363us/step - loss: 0.0381 - acc: 0.9911
Training of node 43 complete


Training chain node  44
Shapes:
 X = (673, 1044) 
 Y = (673, 1)
Epoch 1/5
673/673 [==============================] - 6s 9ms/step - loss: 0.1943 - acc: 0.9747
Epoch 2/5
673/673 [==============================] - 0s 313us/step - loss: 0.1052 - acc: 0.9822
Epoch 3/5
673/673 [==============================] - 0s 353us/step - loss: 0.0570 - acc: 0.9822
Epoch 4/5
673/673 [==============================] - 0s 333us/step - loss: 0.0404 - acc: 0.9822
Epoch 5/5
673/673 [==============================] - 0s 350us/step - loss: 0.0314 - acc: 0.9822
Training of node 44 complete


Training chain node  4

In [13]:
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_fw_bin = output_fw.round().astype(int)
        output_rev = node2.predict(X_feed_re)
        output_rev_bin = output_rev.round().astype(int)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw_bin,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev_bin,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (579, 1001)
Getting op from chain node  1
Shapes:
 X = (579, 1002)
Getting op from chain node  2
Shapes:
 X = (579, 1003)
Getting op from chain node  3
Shapes:
 X = (579, 1004)
Getting op from chain node  4
Shapes:
 X = (579, 1005)
Getting op from chain node  5
Shapes:
 X = (579, 1006)
Getting op from chain node  6
Shapes:
 X = (579, 1007)
Getting op from chain node  7
Shapes:
 X = (579, 1008)
Getting op from chain node  8
Shapes:
 X = (579, 1009)
Getting op from chain node  9
Shapes:
 X = (579, 1010)
Getting op from chain node  10
Shapes:
 X = (579, 1011)
Getting op from chain node  11
Shapes:
 X = (579, 1012)
Getting op from chain node  12
Shapes:
 X = (579, 1013)
Getting op from chain node  13
Shapes:
 X = (579, 1014)
Getting op from chain node  14
Shapes:
 X = (579, 1015)
Getting op from chain node  15
Shapes:
 X = (579, 1016)
Getting op from chain node  16
Shapes:
 X = (579, 1017)
Getting op from chain node  17
Shapes:
 X 

In [14]:
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [15]:
#Adding all 5 results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2)).round().astype(int)

In [16]:
from sklearn.metrics import accuracy_score,hamming_loss, log_loss, f1_score

In [17]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("Hamming accuracy = ",hamming_accuracy_score(y_test,predictions))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions))
print("F1 score = ",f1_score(y_test,predictions,average='macro'))

Accuracy =  0.155440414508
Hamming accuracy =  0.450178065074
Hamming loss =  0.04506794408055528
Log loss =  59.7326737875
F1 score =  0.12306754899
